In [1]:
import pickle
import pandas as pd
import csv
import os
import random
from pymongo import MongoClient
from freq import get_schemes

In [39]:
def get_articles_per_schemes(collName, collection, schemes):
    # (str, pymongo.collection.Collection, dict) -> dict
    stats = {}
    for scheme_name, keywords_list in schemes.items():
        keywords = '|'.join(keywords_list)
        scheme_articles = collection.find({'$and':[{'text': {'$regex': keywords, '$options': 'i'}}]},
                            no_cursor_timeout=True)
        N_scheme_articles = collection.find({'$and':[{'text': {'$regex': keywords, '$options': 'i'}}]},
                            no_cursor_timeout=True).count()
        rr = list(set(random.sample(range(0, N_scheme_articles), min(N_scheme_articles, 20))))
        rr.sort()
        stats[scheme_name] = []
        idx = 0
        for i, art in enumerate(scheme_articles):
            if(rr[idx] == i):
                stats[scheme_name].append(art['text'])
                idx+=1
                if idx == len(rr):
                    break
    return stats

In [40]:
schemes_path = 'schemes/'
schemes_clubbed = get_schemes(schemes_path)

client = MongoClient('localhost', 27017)
db = client['media-db2']

all_stats = {}
for coll in schemes_clubbed:
    collName = coll + '_schemes'
    collection = db[collName]
    scheme_stat = get_articles_per_schemes(collName, collection, schemes_clubbed[coll])
    all_stats[coll] = scheme_stat

/home/ictd/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if __name__ == '__main__':


In [41]:
# for k, v in all_stats.items():
#     print('*****************'+k+'*****************')
#     scheme_stat = v
#     for key, value in scheme_stat.items():
#         print('-----------'+key+'-----------')
#         print(len(value))
#         print()

*****************industrialization*****************
-----------Prime Minister Employment Generation Programme and Other Credit Support Schemes-----------
20

-----------Development of Khadi, Village and Coir Industries-----------
20

-----------Technology Upgradation and Quality Certification-----------
20

-----------Marketing Promotion Schemes-----------
20

-----------Entrepreneurship and skill Development Programme-----------
20

-----------Infrastructure Development Programme-----------
1

-----------Scheme of Surveys, Studies and Policy Research-----------
0

-----------Scheme of Information, Education and Communication-----------
0

-----------North East policy-----------
20

-----------India International Convention and Expo Centre-----------
1

-----------Project Based Support to Autonomous Institutions-----------
0

-----------Standup India-----------
20

-----------National SC-ST-HUB-----------
20

-----------Textile Industry-----------
20

-----------Handloom industry------

In [42]:
# def load():
#     with open('articles.pkl', 'rb') as file:
#         myvar = pickle.load(file)
#         return myvar

In [36]:
# with open('articles.pkl', 'wb') as file:
#         pickle.dump(all_stats, file)
# articles = load()

In [52]:
anotation_path = 'anotate/'
if not os.path.exists(anotation_path):
    os.mkdir(anotation_path)
for scheme, per_scheme in all_stats.items():
    writer = pd.ExcelWriter(anotation_path+scheme+'.xlsx', engine='xlsxwriter')
    df = [None]*len(per_scheme.keys())
    for i, group_per_scheme in enumerate(per_scheme):
        df[i]= pd.DataFrame(per_scheme[group_per_scheme])
        if(len(group_per_scheme) > 31):
            group_per_scheme = ''.join([name[0] for name in group_per_scheme.split()])
        df[i].to_excel(writer, sheet_name=group_per_scheme)
    writer.save()